# Lab 5 - Predicting Midterm Elections
A lab done by both:

Kaden Franklin & 

https://hendrix-cs.github.io/csci285/labs/election.html

In [118]:
# Import pandas
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import math
# Import seaborn
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt


# Apply the default theme
sns.set_theme()


# Step 1: Downloading Polling Data

Polling data retrieved from :

https://projects.fivethirtyeight.com/2022-election-forecast/senate/

This file holds all the senate polls for each state from the last two years. 

Load this file up in a Jupyter notebook as a Pandas DataFrame.

Examine its shape and features and produce a data dictionary explaining each column and what a row of data represents.


In [119]:
poll = pd.read_csv("polls/senate_polls.csv")
poll.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,ranked_choice_reallocated,party,answer,candidate_id,candidate_name,pct
0,81378,218,Hill Research Consultants,1910,Put Utah First,Hill Research Consultants,140,Hill Research Consultants,B/C,NaN,Utah,10/29/22,10/30/22,NaN,NaN,NaN,164443,500,lv,NaN,lv,NaN,10/31/22 16:34,NaN,https://twitter.com/murphymike/status/15871619...,NaN,False,IND,8948,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,REP,Lee,28827,Mike Lee,47.0
1,81378,218,Hill Research Consultants,1910,Put Utah First,Hill Research Consultants,140,Hill Research Consultants,B/C,NaN,Utah,10/29/22,10/30/22,NaN,NaN,NaN,164443,500,lv,NaN,lv,NaN,10/31/22 16:34,NaN,https://twitter.com/murphymike/status/15871619...,NaN,False,IND,8948,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,IND,McMullin,28829,Evan McMullin,46.0
2,81401,1708,Phillips Academy,1781,Abbot Academy Fund,Phillips Academy,699,Phillips Academy,NaN,IVR/Text,Arizona,10/29/22,10/30/22,NaN,NaN,NaN,164491,985,lv,NaN,lv,NaN,11/1/22 09:27,NaN,https://andoverpoll.com/2022/11/01/arizona-dec...,NaN,False,NaN,8919,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,DEM,Kelly,20633,Mark Kelly,47.2
3,81401,1708,Phillips Academy,1781,Abbot Academy Fund,Phillips Academy,699,Phillips Academy,NaN,IVR/Text,Arizona,10/29/22,10/30/22,NaN,NaN,NaN,164491,985,lv,NaN,lv,NaN,11/1/22 09:27,NaN,https://andoverpoll.com/2022/11/01/arizona-dec...,NaN,False,NaN,8919,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,REP,Masters,28668,Blake Masters,46.6
4,81401,1708,Phillips Academy,1781,Abbot Academy Fund,Phillips Academy,699,Phillips Academy,NaN,IVR/Text,Arizona,10/29/22,10/30/22,NaN,NaN,NaN,164491,985,lv,NaN,lv,NaN,11/1/22 09:27,NaN,https://andoverpoll.com/2022/11/01/arizona-dec...,NaN,False,NaN,8919,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,LIB,Victor,30075,Marc Victor,1.7


In [121]:
poll.shape

(1709, 42)

In [122]:
poll.dtypes

poll_id                        int64
pollster_id                    int64
pollster                      object
sponsor_ids                   object
sponsors                      object
display_name                  object
pollster_rating_id             int64
pollster_rating_name          object
fte_grade                     object
methodology                   object
state                         object
start_date                    object
end_date                      object
sponsor_candidate_id         float64
sponsor_candidate             object
sponsor_candidate_party       object
question_id                    int64
sample_size                    int64
population                    object
subpopulation                float64
population_full               object
tracking                      object
created_at                    object
notes                         object
url                           object
source                       float64
internal                        bool
p

# Data Dictionary

|Feature | DataType | Description|
|:--------|:----------|:------------|
| Pollster | object | Name of the organization doing the polling |
| Methodology | object | Form of query (phone call/ in person/ email...) |
| State | object | The state in which the election is occuring |
| Created At | object(datetime)  | Date describing when the poll occured |
| Cycle | int64 | The year/cycle for which the eletion is occuring |
| Office Type | object | The governmental body for which the seat up for election resides in |
| Seat Name | object | The class or periodical for which the seat resides in |
| Election Date | object | The date on which the election will occur |
| Stage | object | The type of election which is occuring (general/runoff) |
| Party | object | The political party in which the politician belongs to |
| Answer | object | The expected answer to the poll (Politicians last name) |
| Candidate Name | object | The name of the candidate being polled for |
| PCT | float64 | Percent of people who selected this choice (Out of 100) |

# Step 2: Preparing the polling data for analysis

Our goal is to predict the probability that Democrats will win key swing states and retain control of the Senate after the November 8th mid-term elections. We will look at the following races,

Nevada (CATHERINE CORTEZ MASTO (D) vs. Adam Laxalt (R))

Pennsylvania (Mehmet Oz (R) vs. John Fetterman (D))

Georgia (RAPHAEL WARNOCK (D) vs. Herschel Walker (R))

Wisconsin (Sen. Ron Johnson, (R) vs. Lt. Gov. Mandela Barnes (D))
Arizona (MARK KELLY (D) vs. Blake Masters (R))

Colorado (MICHAEL BENNET (D) vs. Joe O’Dea (R))

Florida (MARCO RUBIO (R) vs. Val Demings (D))

New Hampshire (MAGGIE HASSAN (D) vs. Don Bolduc (R))

North Carolina (Ted Budd (R) vs. Cheri Beasley (D))

Ohio (J.D. Vance (R) vs. Tim Ryan (D))

For our first assumption, we will only use the most recent poll for each state. 

Format the created_at column of the polling DataFrame as a datetime and then select only the poll with the latest value for each state.

If there are multiple polls entered on the same date, select the one with the larger sample size.

In [123]:
poll['created_at'] = pd.to_datetime(poll['created_at'])

In [124]:
poll['created_at']

0      2022-10-31 16:34:00
1      2022-10-31 16:34:00
2      2022-11-01 09:27:00
3      2022-11-01 09:27:00
4      2022-11-01 09:27:00
               ...        
1704   2021-02-25 09:37:00
1705   2021-01-20 13:38:00
1706   2021-01-20 13:38:00
1707   2021-01-27 13:16:00
1708   2021-01-27 13:16:00
Name: created_at, Length: 1709, dtype: datetime64[ns]

In [125]:
NV_DF = poll.loc[poll["state"] == 'Nevada']

In [126]:
NV_DF.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,ranked_choice_reallocated,party,answer,candidate_id,candidate_name,pct
72,81377,1293,OH Predictive Insights / MBQF,944,The Nevada Independent,OH Predictive Insights,235,OH Predictive Insights,B/C,Live Phone/Text,Nevada,10/24/22,10/27/22,NaN,NaN,NaN,164442,600,lv,NaN,lv,NaN,2022-10-31 16:33:00,NaN,https://thenevadaindependent.com/article/poll-...,NaN,False,NaN,8937,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,DEM,Cortez Masto,28593,Catherine Cortez Masto,43.0
73,81377,1293,OH Predictive Insights / MBQF,944,The Nevada Independent,OH Predictive Insights,235,OH Predictive Insights,B/C,Live Phone/Text,Nevada,10/24/22,10/27/22,NaN,NaN,NaN,164442,600,lv,NaN,lv,NaN,2022-10-31 16:33:00,NaN,https://thenevadaindependent.com/article/poll-...,NaN,False,NaN,8937,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,REP,Laxalt,28594,Adam Paul Laxalt,41.0
87,81312,1302,Echelon Insights,NaN,NaN,Echelon Insights,407,Echelon Insights,B/C,Live Phone/Text,Nevada,10/24/22,10/26/22,NaN,NaN,NaN,164281,500,lv,NaN,lv,NaN,2022-10-27 13:40:00,NaN,https://echeloninsights.com/in-the-news/midter...,NaN,False,NaN,8937,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,DEM,Cortez Masto,28593,Catherine Cortez Masto,45.0
88,81312,1302,Echelon Insights,NaN,NaN,Echelon Insights,407,Echelon Insights,B/C,Live Phone/Text,Nevada,10/24/22,10/26/22,NaN,NaN,NaN,164281,500,lv,NaN,lv,NaN,2022-10-27 13:40:00,NaN,https://echeloninsights.com/in-the-news/midter...,NaN,False,NaN,8937,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,REP,Laxalt,28594,Adam Paul Laxalt,45.0
89,81312,1302,Echelon Insights,NaN,NaN,Echelon Insights,407,Echelon Insights,B/C,Live Phone/Text,Nevada,10/24/22,10/26/22,NaN,NaN,NaN,164281,500,lv,NaN,lv,NaN,2022-10-27 13:40:00,NaN,https://echeloninsights.com/in-the-news/midter...,NaN,False,NaN,8937,2022,U.S. Senate,2,Class III,11/8/22,general,False,False,LIB,Scott,29736,Neil Scott,1.0


If one was estute in observation, they might notice that the first entry in our dataframe is also the most recent.

The top two rows are for the most recent poll from the republican and demcrat parties, thus we will select the DEM, as it will be used further on in the lab

In [127]:
Nev = NV_DF.iloc[0]

In [128]:
Nev['pct']

43.0

In these polls, we will need the data in the sample_size, party, and pct columns. 

The sample_size is how many people were surveyed in this poll, the party lists the choice offered 

(Note: answer is actually the choice offered, but since we are rolling this up to political parties it makes sense to work with the party column)

, and the pct is the percent of people who selected this choice. 

Divide the value in pct by 100, so it is a number between 0 and 1.


In [129]:
Nev['pct'] = Nev['pct']/100

C:\Users\kaden\AppData\Local\Temp\ipykernel_14152\388663704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Nev['pct'] = Nev['pct']/100


In [130]:
Nev['pct']

0.43

These choices don’t always add up to 100%, some small percentage of voters remain undecided or unwilling to offer their choice, and there are small margins for Libertarian, Green, and other party candidates. 

For our second assumption, to help determine the plurality winner for each state, we will say that people are either voting DEM, or voting against DEM. 

Therefore, we can select only those rows with the party equal to DEM.


# Step 3: Generating probability distributions

Polls based on a sample of the whole population include a margin of error.

To determine if DEMs will win a particular state, we will generate a random number using a normal distribution, with a mean of pct for that state’s poll. 

To account for the margin of error, we will use the following formula to calculate the standard deviation for the poll, which is based on the sample_size for that state.

stdv = 1 / (2 * math.sqrt(sample_size))

In [131]:
stdv = 1/ (2 * math.sqrt(Nev['sample_size']))

# Step 4: Simulation

We now have enough data to simulate the elections. 

For each seat, generate a percentage of voters choosing DEM. 

If this generated value is greater than 0.5, then award the seat to the Democrats.

Run the election simulation 40,000 times and collect the results of each simulation.

Plot the (binary) outcomes (e.g. “won” or “lost”) for each state using a bar chart.

In [132]:
# Generate a random percentage chance of DEM winning

prob = []
try_ = 40000

for i in range(try_):
    


IndentationError: expected an indented block (1688512949.py, line 7)

In [133]:
hist = sns.displot(
    data=df,
    x='probability',
    kind='hist',
    height=5,
    aspect=1.5,
)

NameError: name 'df' is not defined

# Step 5: Conclusion

Download current U.S. Senate data. The file should be called “1976-2020-senate.csv”. 

Load this file up in a Jupyter notebook as a Pandas DataFrame.

Examine its shape and features and produce a data dictionary explaining each column and what a row of data represents.

Using the data in this file, and the results of your Monte Carlo simulations, clearly state your predicted 2022 mid-term election outcomes.

Will the Democrats maintain control of the U.S. Senate?

How closely do your results match the predictions at FiveThirtyEight? 

What could explain any differences you see? What other assumptions did we implicitly make in our simulation?

In [134]:
poll_2 = pd.read_csv("polls/senate_polls_historical.csv")
poll_2.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,ranked_choice_reallocated,party,answer,candidate_id,candidate_name,pct
0,73993,1528,AtlasIntel,NaN,NaN,AtlasIntel,546.0,AtlasIntel,A,Live Phone/Online,Georgia,1/2/21,1/4/21,NaN,NaN,NaN,138823,857.0,lv,NaN,lv,NaN,1/4/21 19:35,NaN,https://projects.fivethirtyeight.com/polls/202...,538,False,NaN,8737,2020,U.S. Senate,0,Class II,1/5/21,runoff,False,False,DEM,Ossoff,14567,Jon Ossoff,51.3
1,73993,1528,AtlasIntel,NaN,NaN,AtlasIntel,546.0,AtlasIntel,A,Live Phone/Online,Georgia,1/2/21,1/4/21,NaN,NaN,NaN,138823,857.0,lv,NaN,lv,NaN,1/4/21 19:35,NaN,https://projects.fivethirtyeight.com/polls/202...,538,False,NaN,8737,2020,U.S. Senate,0,Class II,1/5/21,runoff,False,False,REP,Perdue,14568,David A. Perdue,46.8
2,73993,1528,AtlasIntel,NaN,NaN,AtlasIntel,546.0,AtlasIntel,A,Live Phone/Online,Georgia,1/2/21,1/4/21,NaN,NaN,NaN,138824,857.0,lv,NaN,lv,NaN,1/4/21 19:35,NaN,https://projects.fivethirtyeight.com/polls/202...,538,False,NaN,7781,2020,U.S. Senate,2,Class III,1/5/21,runoff,False,False,DEM,Warnock,14489,Raphael Warnock,50.8
3,73993,1528,AtlasIntel,NaN,NaN,AtlasIntel,546.0,AtlasIntel,A,Live Phone/Online,Georgia,1/2/21,1/4/21,NaN,NaN,NaN,138824,857.0,lv,NaN,lv,NaN,1/4/21 19:35,NaN,https://projects.fivethirtyeight.com/polls/202...,538,False,NaN,7781,2020,U.S. Senate,2,Class III,1/5/21,runoff,False,False,REP,Loeffler,13671,Kelly Loeffler,46.9
4,74006,1250,Trafalgar Group,NaN,NaN,Trafalgar Group,338.0,Trafalgar Group,A-,IVR/Online/Text,Georgia,1/2/21,1/4/21,NaN,NaN,NaN,138845,1056.0,lv,NaN,lv,NaN,1/5/21 09:57,NaN,https://www.thetrafalgargroup.org/news/ga-sen-...,538,False,REP,8737,2020,U.S. Senate,0,Class II,1/5/21,runoff,False,False,DEM,Ossoff,14567,Jon Ossoff,49.4


In [ ]:
poll_2.dtypes

# Data Dictionary

|Feature | DataType | Description|
|:--------|:----------|:------------|
| Pollster | object | Name of the organization doing the polling |
| Methodology | object | Form of query (phone call/ in person/ email...) |
| State | object | The state in which the election is occuring |
| Created At | object | Date describing when the poll occured |
| Cycle | int64 | The year/cycle for which the eletion is occuring |
| Office Type | object | The governmental body for which the seat up for election resides in |
| Seat Name | object | The class or periodical for which the seat resides in |
| Election Date | object | The date on which the election will occur |
| Stage | object | The type of election which is occuring (general/runoff) |
| Party | object | The political party in which the politician belongs to |
| Answer | object | The expected answer to the poll (Politicians last name) |
| Candidate Name | object | The name of the candidate being polled for |
| PCT | float64 | Percent of people who selected this choice (Out of 100) |